In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import pickle

evaluation = False
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"
    

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderFullDump
Dataset full loaded


In [2]:
#album
icm = dr.getICM_csr(albums=True,artists=False,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
r_al = urm*s.T
r_al = normalize(r_al, axis=0)
if evaluation: print dr.evaluateMAP(r_al,verbose=False)
r_al = dr.reduceRM(r_al)

In [3]:
#artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

In [4]:
#album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
r_al_ar = urm*s.T #or s
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_al_ar,verbose=False)
r_al_ar = dr.reduceRM(r_al_ar)

In [5]:
#tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
icm = normalize(icm,axis=1)   # little improvment
model = BM25Recommender(K = 15)
model.fit(icm)
s = model.similarity
r_tag = urm*s
r_tag = normalize(r_tag, axis=0)
if evaluation: print dr.evaluateMAP(r_tag, verbose = False)
r_tag = dr.reduceRM(r_tag)

In [6]:
#cf ub
model = BM25Recommender(K=60)
model.fit(urm)
s = model.similarity
r_cfub = s.T * urm
r_cfub = normalize(r_cfub, axis=1)
if evaluation: print dr.evaluateMAP(r_cfub, verbose = False)
r_cfub = dr.reduceRM(r_cfub)

In [7]:
#cf ib
model = BM25Recommender(K=100)
model.fit(urm.T)
s = model.similarity
r_cfib = urm * s.T
r_cfib = normalize(r_cfib, axis=0)
if evaluation: print dr.evaluateMAP(r_cfib, verbose = False)
r_cfib = dr.reduceRM(r_cfib)

In [8]:
r_bpr = dr.reduceRM(urm * dr.load_sparse_csr(filesslim))

if evaluation: print dr.evaluateMAP(r_bpr, verbose=False)

In [9]:
#############start with the mix#################

In [10]:
######### cb

In [31]:
r_cb1 = r_al_ar + 0.25*r_tag 
if evaluation: print dr.evaluateMAP(r_cb1, verbose = False)

In [32]:
#not used atm
#r_totcb = normalize(r_cb1,axis=1) + 0.2* r_al + 0.15 * r_ar
#if evaluation: print dr.evaluateMAP(r_totcb, verbose = False)

In [33]:
########## cf

In [34]:
#total cf
r_totcf = 1.25*r_cfib + r_cfub
if evaluation: print dr.evaluateMAP(r_totcf, verbose = False)

In [35]:
########## all

In [36]:
r_tot = r_cb1 + 1.80 * r_totcf
if evaluation: print dr.evaluateMAP(r_tot,verbose = False)

In [38]:
#not used atm
#r_tot2 = r_tot + 5* r_bpr

#if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

In [37]:
rec = dr.getAllRecommendations(r_tot)
dr.buildSubmissionFile(recommendation=rec, file_name="ci_si_prova_2_tag25cf190")

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommendations done
Creating submission file in: ./Submissions/ci_si_prova_2_tag25cf190
Done


In [ ]:
## actually not used

In [8]:
#collaborative filtering item based asym
m_cf_ib = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.30, q=1, m=0)
s = m_cf_ib.s
r_cfib_asy = urm*s
if evaluation: print dr.evaluateMAP(r_cfib_asy, verbose=False)
# reduce matrix
r_cfib_asy_more = normalize(r_cfib,axis=1) #4.75
r_cfib_asy_less = normalize(r_cfib,axis=0)#2.75 #######axis 0 better in sum

r_cfib_asy = dr.reduceRM(r_cfib_asy)
r_cfib_asy_more = dr.reduceRM(r_cfib_asy_more)
r_cfib_asy_less = dr.reduceRM(r_cfib_asy_less)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done
0.0720835555556
